In [2]:
import glob as glob
import pandas as pd
import os
import shutil
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import sem
#from statsmodels.stats.anova import AnovaRM
#import scikit_posthocs as sp
#import statsmodels.formula.api as smf

In [3]:
from scipy import stats

In [4]:
import statsmodels.api as sm

In [5]:
sns.__version__

'0.12.2'

### load started frame info

In [6]:
disk_path="put the disk directory here"

In [7]:
parent_folder=disk_path + "\\CPT_TOT_DS_Stage_3_2nd\\VFA"

In [7]:
frame_cal=pd.read_excel(parent_folder+"\\started frame_2.xlsx")
# frame_cal['vfa_session']=frame_cal['session name'].str[1:] + ".csv"
# frame_cal['group_id']=frame_cal['group_id'].str[1:]
# frame_cal["n_fps"]=60
# frame_cal.to_csv(r"D:\CPT_TOT_DS_Stage_3_2nd\VFA\started frame_2.xlsx", index=False)

### grasp before event time window orientation info

In [8]:
# load edit files
path_list=disk_path+"\\CPT_TOT_DS_Stage_3_2nd\\VFA\\dlc_output\\results\\threshold=3\\temp_2_V2"
os.chdir(path_list)

save_path=disk_path + "\\CPT_TOT_DS_Stage_3_2nd\\VFA\\amph_ori_LH_V2"
    
code_file_name=frame_cal["vfa_session"].tolist()
nfps=frame_cal["n_fps"].tolist()
event_list=["Hits - #", "Misses - #" , "Correct Rejections - #", "Mistakes - #" ]
bin_list=["15bin_1", "15bin_2","15bin_3","15bin_4","15bin_5","15bin_6"]

for n, b in zip(code_file_name, nfps):    
    file=pd.read_csv(n)
    df_=file[file["event"].isin(["Stim Onset - End ITI", "Hits - #", "Misses - #" , "Correct Rejections - #", "Mistakes - #" ])].copy()

    hit_index=[]
    miss_index=[]
    mistake_index=[]
    corr_reject_index=[]
    for e in range(len(df_)-1):
        if df_.iloc[e]["event"]=="Stim Onset - End ITI" and df_.iloc[e+1]["event"]=="Hits - #":
            hit_index.append(df_.iloc[[e,e+1]]["sec_cal"].to_list())
        if df_.iloc[e]["event"]=="Stim Onset - End ITI" and df_.iloc[e+1]["event"]=="Misses - #":
            miss_index.append(df_.iloc[[e,e+1]]["sec_cal"].to_list())
        if df_.iloc[e]["event"]=="Stim Onset - End ITI" and df_.iloc[e+1]["event"]=="Mistakes - #":
            mistake_index.append(df_.iloc[[e,e+1]]["sec_cal"].to_list())
        if df_.iloc[e]["event"]=="Stim Onset - End ITI" and df_.iloc[e+1]["event"]=="Correct Rejections - #":
            corr_reject_index.append(df_.iloc[[e,e+1]]["sec_cal"].to_list())

    hit=[]
    miss=[]
    mistake=[]
    corr_reject=[]        
    for l, c in zip([hit_index,miss_index,mistake_index,corr_reject_index],
                    ["hit", "miss", "mistake", "corr_reject"]):
        for f, a in zip(l, range(1, len(l)+1)):
            b=file[(file["sec_cal"] >= f[0]) & (file["sec_cal"] <= f[1])].copy()
            b["trial_number"]=a
            b["event_type"]=c

            if c=="hit":
                hit.append(b)            
            elif c=="miss":
                miss.append(b)
            elif c=="mistake":
                mistake.append(b)
            else:
                corr_reject.append(b)            
    hit=pd.concat(hit, ignore_index=True)
    miss=pd.concat(miss, ignore_index=True)
    mistake=pd.concat(mistake, ignore_index=True)
    corr_reject=pd.concat(corr_reject, ignore_index=True)

    #add each bin's trial number into df according to event type
    event_list=["Hits - #", "Misses - #" ,"Mistakes - #", "Correct Rejections - #"]
    df=[hit, miss, mistake, corr_reject]
    for x, y in zip(event_list, df): 
        bin_n=[]
        bin_condition=[(y["15min_bin"]=="15bin_1"), (y["15min_bin"]=="15bin_2"),
                   (y["15min_bin"]=="15bin_3"),(y["15min_bin"]=="15bin_4"),
                   (y["15min_bin"]=="15bin_5"),(y["15min_bin"]=="15bin_6")]
        for z in bin_condition:
            bin_n.append(len(y[(y["event"]==x) & z]))
        y["bin_trial_n"]=np.select(bin_condition, bin_n)
    #combine four different event into individual file
    df=pd.concat([hit, miss, mistake, corr_reject], ignore_index=True)
    df.to_csv(save_path + "\\%s"%(n), index=False)

C:\Users\ye.li\AppData\Local\Temp\ipykernel_3964\623070062.py:13: DtypeWarning: Columns (1,36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  file=pd.read_csv(n)
C:\Users\ye.li\AppData\Local\Temp\ipykernel_3964\623070062.py:13: DtypeWarning: Columns (1,36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  file=pd.read_csv(n)
C:\Users\ye.li\AppData\Local\Temp\ipykernel_3964\623070062.py:13: DtypeWarning: Columns (1,36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  file=pd.read_csv(n)
C:\Users\ye.li\AppData\Local\Temp\ipykernel_3964\623070062.py:13: DtypeWarning: Columns (1,36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  file=pd.read_csv(n)
C:\Users\ye.li\AppData\Local\Temp\ipykernel_3964\623070062.py:13: DtypeWarning: Columns (1,36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  file=pd.read_csv(n)
C:\Users\ye.li\

### calculate orientation duration between stim and event

In [9]:
# load edit files
fromfolder=disk_path + "\\CPT_TOT_DS_Stage_3_2nd\\VFA\\amph_ori_LH_V2"
os.chdir(fromfolder)

save_path=disk_path+"\\CPT_TOT_DS_Stage_3_2nd\\VFA\\amph_ori_LH_V2\\15_amph_ori_LH_V2"
if os.path.isdir(save_path) == False:
    os.mkdir(save_path)
    
code_file_name=frame_cal["vfa_session"].tolist()
nfps=frame_cal["n_fps"].tolist()

for i,h in zip(code_file_name, nfps):
    file=pd.read_csv(i)
    df=file.groupby(by=["15min_bin", "event_type", "blind_or_not"], as_index=False).agg(
        bin_frame_sum=('index', 'count'),
        bin_trial_n_sum=('bin_trial_n', 'mean')).copy()
    df["cohort"]="coh2"
#     df["dt_id"]=n[:-d]
#     df["animal_id"]=df["dt_id"][1][11:]
#     df["id_cohort"]=df["animal_id"]+ "-" + df["cohort"]       
    df["duration"]=df['bin_frame_sum']/h
    df["ori_mean"]=df["duration"]/df["bin_trial_n_sum"]
    df.to_csv(save_path + "\\%s"%(i), index=False)

### plot 15 min plot for orientation duration

In [10]:
def normal(mean, std, histmax=False, color="black"):
    x = np.linspace(mean-4*std, mean+4*std, 200)
    p = stats.norm.pdf(x, mean, std)
    if histmax:
        p = p*histmax/max(p)
    z = plt.plot(x, p, color, linewidth=2)

In [11]:
path_list=disk_path+"\\CPT_TOT_DS_Stage_3_2nd\\VFA\\amph_ori_LH_V2\\15_amph_ori_LH_V2"
df_list=glob.glob(path_list+"\\*.csv")

os.chdir(path_list)
sess_name=glob.glob("*.csv")

save_path=disk_path+"\\CPT_TOT_manuscript\\data processing\\fig\\15_amph_ori_LH_V2"   

if os.path.isdir(save_path ) == False:
    os.mkdir(save_path)
    
df=pd.DataFrame()
for i, a in zip(df_list, sess_name):
    df_=pd.read_csv(i)
    if len(df_)<8:
        print(i)
    df_["session_name"]=a[:-4]
    df=pd.concat([df, df_], ignore_index=True)
    
dosage_cond=[(df["session_name"].str.contains("AMPH-vehicle")),
        (df["session_name"].str.contains("AMPH-0.3mpk")),
            (df["session_name"].str.contains("AMPH-0.6mpk")),
            (df["session_name"].str.contains("AMPH-1mpk")),]
dosage_list=['AMPH-vehicle','AMPH-0.3mpk', 'AMPH-0.6mpk', 'AMPH-1mpk']  
df['dosage']=np.select(dosage_cond, dosage_list)
df["dt_id"]=df["session_name"]

for h in range(len(df)):
    df.loc[h, "dt_id"]=df.loc[h,'dt_id'].replace("-" + df.loc[h,'dosage'], "")
    
df["animal_id"]=df["dt_id"].str[11:]
df["id_cohort"]=df["animal_id"]+ "-" + df["cohort"]

df['gander']=np.where((df["session_name"].str.contains("-A-")) | (df["session_name"].str.contains("-B-")), 'female', 'male')
df['group_id']=df['animal_id'].apply(lambda x:x[0])
df["blind_or_not"].replace({"not_blind":"oriented"}, inplace=True)

event=["hit", "miss", "mistake", "corr_reject"]
gander=["male", "female"]
color_list=['royalblue', 'coral']

for i, p in zip(gander, color_list) :
    for j in event:   
        for m,n,o in zip(["oriented", "blind"], ["o", "x"],['-', '-.']):
            title="%s_amph_effect_on_%s_duration_of_%s_in_LH"%(i,m, j)
            t=df[(df["gander"]==i) & (df["event_type"]==j) 
                 & (df["15min_bin"]!="after_session") 
                 & (df["15min_bin"]!="before_session")
                & (df["blind_or_not"]==m)].copy()
            t=t.rename(columns={"15min_bin": "min_15bin"})

            plot=sns.pointplot(data=t, x="min_15bin", y="ori_mean", hue="dosage",  
                               hue_order=["AMPH-vehicle", "AMPH-0.3mpk", "AMPH-0.6mpk","AMPH-1mpk"], 
                               palette=["black", "green", "royalblue", "red"], 
                               linestyles=[o,o,o,o], markers=[n,n,n,n], capsize=.01, errorbar=('ci', 68))
            plot.set(xlabel="")
            plot.set(ylabel="duration_mean")
            plt.title(title)
            plt.xticks(rotation = 15, ha='right')
        #     h = plt.gca().get_lines()
        #     lg = plt.legend(handles=h, labels=['oriented', 'blind'], loc='best')
            plt.legend(bbox_to_anchor=(1.35, 1),borderaxespad=0)
            t.to_csv(save_path+"\\"+title+".csv")
            
            plt.savefig(save_path+"\%s.png"%(title), dpi=800, bbox_inches="tight")
            plt.close()



C:\Users\ye.li\AppData\Local\anaconda3\Lib\site-packages\seaborn\categorical.py:1728: UserWarning: You passed a edgecolor/edgecolors ((0.0, 0.0, 0.0)) for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  ax.scatter(x, y, label=hue_level,
C:\Users\ye.li\AppData\Local\anaconda3\Lib\site-packages\seaborn\categorical.py:1728: UserWarning: You passed a edgecolor/edgecolors ((0.0, 0.5019607843137255, 0.0)) for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  ax.scatter(x, y, label=hue_level,
C:\Users\ye.li\AppData\Local\anaconda3\Lib\site-packages\seaborn\categorical.py:1728: UserWarning: You passed a edgecolor/edgecolors ((0.2549019607843137, 0.4117647058823529, 0.8823529411764706)) for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  ax.sc

C:\Users\ye.li\AppData\Local\anaconda3\Lib\site-packages\seaborn\categorical.py:1728: UserWarning: You passed a edgecolor/edgecolors ((0.0, 0.0, 0.0)) for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  ax.scatter(x, y, label=hue_level,
C:\Users\ye.li\AppData\Local\anaconda3\Lib\site-packages\seaborn\categorical.py:1728: UserWarning: You passed a edgecolor/edgecolors ((0.0, 0.5019607843137255, 0.0)) for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  ax.scatter(x, y, label=hue_level,
C:\Users\ye.li\AppData\Local\anaconda3\Lib\site-packages\seaborn\categorical.py:1728: UserWarning: You passed a edgecolor/edgecolors ((0.2549019607843137, 0.4117647058823529, 0.8823529411764706)) for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  ax.sc

### calculated ori index
##### ori_index: determine LH trial orientation by using frame orientation, as long as one frame in LH is orientated, we determine this frame as orientated frame, if not, we determine the frame as blind

In [12]:
# load edit files
path_list=disk_path + "\\CPT_TOT_DS_Stage_3_2nd\\VFA\\amph_ori_LH_V2"
os.chdir(path_list)

save_path=disk_path + "\\CPT_TOT_DS_Stage_3_2nd\\VFA\\amph_ori_LH_V2\\15_amph_ori_index_LH_V2"
if os.path.isdir(save_path) == False:
    os.mkdir(save_path)
    
save_total=disk_path + "\\CPT_TOT_DS_Stage_3_2nd\\VFA\\amph_ori_LH_V2\\15_amph_ori_index_LH_V2\\total"
if os.path.isdir(save_total) == False:
    os.mkdir(save_total) 
    
code_file_name=frame_cal["vfa_session"].tolist()
nfps=frame_cal["n_fps"].tolist()
event_list=["hit", "miss", "mistake", "corr_reject"]

for i,h in zip(code_file_name, nfps):
    file=pd.read_csv(i)
    
    file["session_name"]=i[:-4]
    df=file.groupby(by=["session_name", "15min_bin", "event_type", "trial_number", "blind_or_not"], as_index=False).count().copy()
    df["cohort"]="coh2"
    df["count_"]=1
    dosage_cond=[(df["session_name"].str.contains("AMPH-vehicle")),(df["session_name"].str.contains("AMPH-0.3mpk")),
            (df["session_name"].str.contains("AMPH-0.6mpk")),(df["session_name"].str.contains("AMPH-1mpk")),]
    dosage_list=['AMPH-vehicle','AMPH-0.3mpk', 'AMPH-0.6mpk', 'AMPH-1mpk']  
    df['dosage']=np.select(dosage_cond, dosage_list)
    
    df["dt_id"]=df["session_name"]
    for n in range(len(df)):
        df.loc[n, "dt_id"]=df.loc[n,'dt_id'].replace("-" + df.loc[n,'dosage'], "")

    df["animal_id"]=df["dt_id"].str[11:]
    df["id_cohort"]=df["animal_id"]+ "-" + df["cohort"]
    df['group_id']=df['animal_id'].apply(lambda x:x[0])
    #display(df)
    df_=pd.pivot_table(df, index=["15min_bin", "event_type", "trial_number", "dt_id", "animal_id", "id_cohort"], 
                      columns=["blind_or_not"], values=["count_"]).reset_index().copy()#, aggfunc="count"

    df_.columns=[''.join(col) for col in df_.columns.values]
    df_["count_blind"] = df_["count_blind"].fillna(0)
    df_["count_oriented"] = df_["count_oriented"].fillna(value=0)
    #display(df_)

    ##### in column "blind_minus_ori_diff", if value ==  -1.0, meaning that trial is blind trial: if value=0 or 1, meaning that trial is oriented trial. 
    ##### the orientation index is calculated by oriented trial divided by trial number in each bin. 
    df_["blind_minus_ori_diff"]=df_["count_oriented"]-df_["count_blind"]         
    df_["bin_trial_n"]=0
    for x in event_list:    
        bin_condition=[(df_["15min_bin"]=="15bin_1") & (df_["event_type"]==x), (df_["15min_bin"]=="15bin_2")  & (df_["event_type"]==x),
                   (df_["15min_bin"]=="15bin_3") & (df_["event_type"]==x), (df_["15min_bin"]=="15bin_4") & (df_["event_type"]==x),
                   (df_["15min_bin"]=="15bin_5") & (df_["event_type"]==x), (df_["15min_bin"]=="15bin_6") & (df_["event_type"]==x)]
        bin_n=[]            
        for z in bin_condition:
            bin_n.append(len(df_[z]))
        df_["bin_trial_n"]=np.select(bin_condition, bin_n, default=df_["bin_trial_n"]) 
    df_["onetozero"]=df_["blind_minus_ori_diff"].replace(1, 0)
    #display(df_)
    df_event=df_.groupby(by=["15min_bin", "event_type", "dt_id", "animal_id", "id_cohort"], as_index=False).agg(
        only_blind_trial_sum=('onetozero', 'sum'), bin_trial_n_sum=('bin_trial_n', 'mean'))        
    df_event["ori_trial"]=df_event["bin_trial_n_sum"]+df_event["only_blind_trial_sum"]
    df_event["ori_index"]=df_event["ori_trial"]/df_event["bin_trial_n_sum"]
    #display(df_)
    df_event.to_csv(save_path + "\\%s"%(i), index=False)
    
    df_total=df_event.groupby(by=["15min_bin", "dt_id", "animal_id", "id_cohort"], as_index=False).sum()
#     df_total=df_.groupby(by=["15min_bin", "dt_id", "animal_id", "id_cohort"], as_index=False).agg(
#         only_blind_trial_sum=('onetozero', 'sum'), bin_trial_n_sum=('bin_trial_n', 'mean'))  
    df_total["ori_index_total"]=df_total["ori_trial"]/df_total["bin_trial_n_sum"]
#     df_total["ori_trial"]=df_total["bin_trial_n_sum"]+df_total["only_blind_trial_sum"]  
    df_total.to_csv(save_total+"\\%s"%(i), index=False)

### plot ori index plot

In [2]:
#seperate event and put dosage into one plot

path_list=disk_path + "\\CPT_TOT_DS_Stage_3_2nd\\VFA\\amph_ori_LH_V2\\15_amph_ori_index_LH_V2"
df_list=glob.glob(path_list+"\\*.csv")

os.chdir(path_list)
sess_name=glob.glob("*.csv")

save_path=disk_path+"\\CPT_TOT_manuscript\\data processing\\fig\\15_amph_ori_index_LH_V2"
if os.path.isdir(save_path ) == False:
    os.mkdir(save_path)
    
df=pd.DataFrame()
for i, a in zip(df_list, sess_name):
    df_=pd.read_csv(i)
    if len(df_)<8:
        print(i)
    df_["session_name"]=a[:-4]
    df=pd.concat([df, df_], ignore_index=True)
    
dosage_cond=[(df["session_name"].str.contains("AMPH-vehicle")),
        (df["session_name"].str.contains("AMPH-0.3mpk")),
            (df["session_name"].str.contains("AMPH-0.6mpk")),
            (df["session_name"].str.contains("AMPH-1mpk")),]
dosage_list=['AMPH-vehicle','AMPH-0.3mpk', 'AMPH-0.6mpk', 'AMPH-1mpk']  
df['dosage']=np.select(dosage_cond, dosage_list)
df["dt_id"]=df["session_name"]
for h in range(len(df)):
    df.loc[h, "dt_id"]=df.loc[h,'dt_id'].replace("-" + df.loc[h,'dosage'], "")
    
df["cohort"]="coh2"    
df["animal_id"]=df["dt_id"].str[11:]
df["id_cohort"]=df["animal_id"]+ "-" + df["cohort"]

df['gander']=np.where((df["session_name"].str.contains("-A-")) | (df["session_name"].str.contains("-B-")), 'female', 'male')
df['group_id']=df['animal_id'].apply(lambda x:x[0])

event=["hit", "miss", "mistake", "corr_reject"]
gander=["male", "female"]
color_list=['royalblue', 'coral']
for i, p in zip(gander, color_list) :
    for j in event:   
        title="%s_amph_effect_on_orientation_index_of_%s_in_LH"%(i, j)
        t=df[(df["gander"]==i) & (df["event_type"]==j) 
             & (df["15min_bin"]!="after_session") 
             & (df["15min_bin"]!="before_session")].copy()
        t=t.rename(columns={"15min_bin": "min_15bin"})

        plot=sns.pointplot(data=t, x="min_15bin", y="ori_index", hue="dosage",  
                           hue_order=["AMPH-vehicle", "AMPH-0.3mpk", "AMPH-0.6mpk","AMPH-1mpk"], 
                           palette=["black", "green", "royalblue", "red"], capsize=.01, errorbar=('ci', 68))
        plot.set(xlabel="")
        plot.set(ylabel="orientation index")
        plt.title(title)
        plt.xticks(rotation = 15, ha='right')
    #     h = plt.gca().get_lines()
    #     lg = plt.legend(handles=h, labels=['oriented', 'blind'], loc='best')
        plt.legend(bbox_to_anchor=(1.35, 1),borderaxespad=0)
        t.to_csv(save_path+"\\"+title+".csv")
        
        plt.savefig(save_path+"\%s.png"%(title), dpi=800, bbox_inches="tight")
        plt.close()


### plot ori index total plot

In [1]:
#seperate event and put dosage into one plot

path_list=disk_path+"\\CPT_TOT_DS_Stage_3_2nd\\VFA\\amph_ori_LH_V2\\15_amph_ori_index_LH_V2\\total"
df_list=glob.glob(path_list+"\\*.csv")

os.chdir(path_list)
sess_name=glob.glob("*.csv")

save_path=disk_path+"\\CPT_TOT_manuscript\\data processing\\fig\\15_amph_ori_index_LH_V2_total"
if os.path.isdir(save_path ) == False:
    os.mkdir(save_path)
    
df=pd.DataFrame()
for i, a in zip(df_list, sess_name):
    df_=pd.read_csv(i)
#     if len(df_)<8:
#         print(i)
    df_["session_name"]=a[:-4]
    df=pd.concat([df, df_], ignore_index=True)
    
dosage_cond=[(df["session_name"].str.contains("AMPH-vehicle")),
        (df["session_name"].str.contains("AMPH-0.3mpk")),
            (df["session_name"].str.contains("AMPH-0.6mpk")),
            (df["session_name"].str.contains("AMPH-1mpk")),]
dosage_list=['AMPH-vehicle','AMPH-0.3mpk', 'AMPH-0.6mpk', 'AMPH-1mpk']  
df['dosage']=np.select(dosage_cond, dosage_list)
df["dt_id"]=df["session_name"]
for h in range(len(df)):
    df.loc[h, "dt_id"]=df.loc[h,'dt_id'].replace("-" + df.loc[h,'dosage'], "")
    
df["cohort"]="coh2"    
df["animal_id"]=df["dt_id"].str[11:]
df["id_cohort"]=df["animal_id"]+ "-" + df["cohort"]

df['gander']=np.where((df["session_name"].str.contains("-A-")) | (df["session_name"].str.contains("-B-")), 'female', 'male')
df['group_id']=df['animal_id'].apply(lambda x:x[0])


gander=["male", "female"]
color_list=['royalblue', 'coral']
for i, p in zip(gander, color_list) :
    title="%s_amph_effect_on_total_orientation_index_in_LH"%(i)
    t=df[(df["gander"]==i) & (df["15min_bin"]!="after_session") 
         & (df["15min_bin"]!="before_session")].copy()
    t=t.rename(columns={"15min_bin": "min_15bin"})

    plot=sns.pointplot(data=t, x="min_15bin", y="ori_index_total", hue="dosage",  
                       hue_order=["AMPH-vehicle", "AMPH-0.3mpk", "AMPH-0.6mpk","AMPH-1mpk"], 
                       palette=["black", "green", "royalblue", "red"], capsize=.01, errorbar=('ci', 68))
    plot.set(xlabel="")
    plot.set(ylabel="orientation index total")
    plt.title(title)
    plt.xticks(rotation = 15, ha='right')
#     h = plt.gca().get_lines()
#     lg = plt.legend(handles=h, labels=['oriented', 'blind'], loc='best')
    plt.legend(bbox_to_anchor=(1.35, 1),borderaxespad=0)
    t.to_csv(save_path+"\\"+title+".csv")

    plt.savefig(save_path+"\%s.png"%(title), dpi=800, bbox_inches="tight")
    plt.close()


## calculate AMPH response rate and passive rate： 
### response rate=($N_{Hit}$+$N_{Mistake}$) / ($N_{Hit}$+$N_{Mistake}$+$N_{Miss}$+$N_{Corr reject}$ )
### passive rate=($N_{Miss}$+$N_{Corr reject}$) / ($N_{Hit}$+$N_{Mistake}$+$N_{Miss}$+$N_{Corr reject}$ )

In [20]:
# load edit files
path=disk_path + "\\CPT_TOT_DS_Stage_3_2nd\\VFA\\amph_ori_LH_V2"
os.chdir(path)

save_path=path+"\\15_response_rate_LH"
if os.path.isdir(save_path) == False:
    os.mkdir(save_path)

df_list=glob.glob("*.csv")
event_list=["Hits - #", "Misses - #", "Mistakes - #", "Correct Rejections - #"]

for n in df_list:
    df=pd.read_csv(n)
    df_=df[df["event"].isin(event_list)][["event","15min_bin", "bin_trial_n"]].copy()
    t=df_.groupby(by=['event','15min_bin'], as_index=False).agg(
        event_count=('event', 'count'),
        bin_trial_n_sum=('bin_trial_n', 'mean'))

    t_=pd.pivot_table(t, index=["15min_bin"],columns=["event"], values=["event_count"]).reset_index()
    
    t_.columns=['_'.join(col) for col in t_.columns.values]
    
    
    #grape back the deleted feature columns
    t_["cohort"]="coh2"
    t_["session_name"]=n[:-4]    
    
    dosage_cond=[(t_["session_name"].str.contains("AMPH-vehicle")),
            (t_["session_name"].str.contains("AMPH-0.3mpk")),
                (t_["session_name"].str.contains("AMPH-0.6mpk")),
                (t_["session_name"].str.contains("AMPH-1mpk")),]
    dosage_list=['AMPH-vehicle','AMPH-0.3mpk', 'AMPH-0.6mpk', 'AMPH-1mpk']  
    t_['dosage']=np.select(dosage_cond, dosage_list)
    
    t_["dt_id"]=t_["session_name"]
    for h in range(len(t_)):
        t_.loc[h, "dt_id"]=t_.loc[h,'dt_id'].replace("-" + t_.loc[h,'dosage'], "")
 
    t_["animal_id"]=t_["dt_id"].str[11:]
    t_["id_cohort"]=t_["animal_id"]+ "-" + t_["cohort"]
    t_['gander']=np.where((t_["session_name"].str.contains("-A-")) | (t_["session_name"].str.contains("-B-")), 'female', 'male')
    t_['group_id']=t_['animal_id'].apply(lambda x:x[0])
    
    t_.rename(columns={'event_count_Correct Rejections - #': 'n_corrj', 'event_count_Hits - #': 'n_hit', 
                       'event_count_Misses - #': 'n_miss', 'event_count_Mistakes - #': 'n_mistake', "15min_bin_": "min_15bin"}, inplace=True)
    
    #calculate response rate and passive rate for each mice
    t_=t_.fillna(0)
    t_['response_rate']=(t_['n_hit'] + t_['n_mistake'])/(t_['n_hit'] + t_['n_mistake'] + t_['n_miss'] + t_['n_corrj'])
    t_['passive_rate']=(t_['n_miss'] + t_['n_corrj'])/(t_['n_hit'] + t_['n_mistake'] + t_['n_miss'] + t_['n_corrj'])
    
    t_.to_csv(save_path + "\\%s_-_%s_response_rate.csv"%(t_["animal_id"][1],t_["dosage"][1]), index=False)

In [81]:
path=disk_path + "\\CPT_TOT_DS_Stage_3_2nd\\VFA\\amph_ori_LH_V2"
rr_path=path+"\\15_response_rate_LH"
rr_df=glob.glob(rr_path+"\\*.csv")

rr=pd.DataFrame()
for i in rr_df:
    df_=pd.read_csv(i)
    rr=pd.concat([rr, df_], ignore_index=True)
rr=rr.drop(["dt_id","group_id", "session_name", "id_cohort"], axis=1)

ori_path=disk_path+"\\CPT_TOT_manuscript\\data processing\\fig\\15_amph_ori_index_LH_V2_total\\"
ori_list=glob.glob(ori_path+"*.csv")

folder="for_publication"
save_path=ori_path+folder
if os.path.isdir(save_path ) == False:
    os.mkdir(save_path) 
    
for i in ori_list:
    ori_df=pd.read_csv(i)
    rr_df=rr[rr["gander"]==ori_df["gander"][2]].copy()
    rr_df.to_csv(save_path+"\\%s_response_rate.csv"%(ori_df["gander"][2]))
    
    
    
    
    if ori_df["gander"][2]=="male":
        plot=sns.pointplot(data=ori_df, x="min_15bin", y="ori_index_total", hue="dosage",  
                       hue_order=["AMPH-vehicle", "AMPH-0.3mpk", "AMPH-0.6mpk","AMPH-1mpk"], dodge=True,
                       palette=["powderblue", "deepskyblue",  "royalblue", "midnightblue"], 
                           linestyles=['-', ':', '-.', '--'],
                           capsize=.15, errwidth=1, errorbar=('ci', 68))
      
    else:
        plot=sns.pointplot(data=ori_df, x="min_15bin", y="ori_index_total", hue="dosage",  
                       hue_order=["AMPH-vehicle", "AMPH-0.3mpk", "AMPH-0.6mpk","AMPH-1mpk"], dodge=True,
                       palette=["peachpuff", "hotpink",  "coral", "maroon"], linestyles=['-', ':', '-.', '--'],
                           capsize=.15, errwidth=1, errorbar=('ci', 68))       

    plot.set(xlabel="")
    plot.set(ylabel="Total orientation index")
    new_labels = ["1", "2", "3", "4", "5", "6"]
    plot.set_xticklabels(new_labels)
    plot.set(xlabel="Bin")
    #plt.ylim(0.80, 1.0 )
    plot.yaxis.set_ticks(np.arange(0.0, 1.1, 0.1))
    #plt.title(title)
    #plt.xticks(rotation = 15, ha='right')
#     h = plt.gca().get_lines()
#     lg = plt.legend(handles=h, labels=['oriented', 'blind'], loc='best')
    plt.legend(bbox_to_anchor=(1.15, 1),borderaxespad=0)
    #df.to_csv(save_path+"\\"+title+".csv")
    plot.spines['right'].set_visible(False)
    plot.spines['top'].set_visible(False)
    
    if ori_df["gander"][2]=="male":
        ax2=plot.twinx()
        bar=sns.barplot(data=rr_df, x="min_15bin", y="response_rate", hue="dosage", capsize=.15, errorbar=('ci', 68), 
                        hue_order=["AMPH-vehicle", "AMPH-0.3mpk", "AMPH-0.6mpk","AMPH-1mpk"],
                         palette=["powderblue", "deepskyblue",  "royalblue", "midnightblue"], 
                        errwidth=1, ax=ax2, linewidth=1.5, edgecolor=".3")
    else:
        ax2=plot.twinx()
        bar=sns.barplot(data=rr_df, x="min_15bin", y="response_rate", hue="dosage", capsize=.15, errorbar=('ci', 68), 
                        hue_order=["AMPH-vehicle", "AMPH-0.3mpk", "AMPH-0.6mpk","AMPH-1mpk"],
                         palette=["peachpuff", "hotpink",  "coral", "maroon"], 
                        errwidth=1, ax=ax2, linewidth=1.5, edgecolor=".3") 
    
    
    bar.set(ylabel="Response rate")
    new_labels = ["1", "2", "3", "4", "5", "6"]
    bar.set_xticklabels(new_labels)
    bar.set(xlabel="Bin")
    plt.ylim(0.0, 0.6)
    bar.spines['top'].set_visible(False)
    plt.legend(bbox_to_anchor=(1.45, 0.7),borderaxespad=0)
    plt.savefig(save_path+"\\%s_total_ori_index_with_rr.png"%(ori_df["gander"][2]), dpi=800, bbox_inches="tight")
    plt.close()


## calculate Total AMPH response rate, passive rate & Total orientation index： 

In [83]:
#total orientation index
ori_path=disk_path + "\\CPT_TOT_manuscript\\data processing\\fig\\15_amph_ori_index_LH_V2_total\\"
ori_list=glob.glob(ori_path+"*.csv")

save_path=disk_path + "\\CPT_TOT_manuscript\\data processing\\fig\\amph_total_ori_index_rr"
if os.path.isdir(save_path ) == False:
    os.mkdir(save_path)
    
ori=pd.DataFrame()
for i in ori_list:
    df_=pd.read_csv(i)
    ori=pd.concat([ori, df_], ignore_index=True)
    
ori_=ori.drop(["dt_id","min_15bin", "cohort", "event_type", "session_name", "id_cohort", "ori_index", "ori_index_total","group_id"], axis=1).copy()
ori_=ori_.groupby(by=["animal_id", "dosage", "gander"], as_index=False).sum()
ori_['ori_index']=ori_['ori_trial']/ori_['bin_trial_n_sum']
ori_.to_csv(save_path+"\\total_amph_ori_index.csv")

path=disk_path + "\\CPT_TOT_DS_Stage_3_2nd\\VFA\\amph_ori_LH_V2"
rr_path=path+"\\15_response_rate_LH"
rr_df=glob.glob(rr_path+"\\*.csv")

rr=pd.DataFrame()
for i in rr_df:
    df_=pd.read_csv(i)
    rr=pd.concat([rr, df_], ignore_index=True)
    
rr_=rr.drop(["dt_id","min_15bin", "cohort",  "session_name", "id_cohort", "response_rate","passive_rate", "group_id"], axis=1).copy()
rr_=rr_.groupby(by=["animal_id", "dosage", "gander"], as_index=False).sum()
rr_['response_rate']=(rr_['n_hit'] + rr_['n_mistake'])/(rr_['n_hit'] + rr_['n_mistake'] + rr_['n_miss'] + rr_['n_corrj'])
rr_['passive_rate']=(rr_['n_miss'] + rr_['n_corrj'])/(rr_['n_hit'] + rr_['n_mistake'] + rr_['n_miss'] + rr_['n_corrj'])
rr_.to_csv(save_path+"\\total_amph_rr.csv")

## create bar plot for overall amph orient index

In [16]:
#seperate event and put dosage into one plot

path_list=disk_path+"\\CPT_TOT_manuscript\\data processing\\fig\\15_amph_ori_index_LH_V2_total"
df_list=glob.glob(path_list+"\\*.csv")

folder='\\bar_plot'
save_path=path_list+folder
if os.path.isdir(save_path ) == False:
    os.mkdir(save_path)
    
for i in df_list:
    if df['gander'][2]=='male':
        x='male'
    else:
        x='female'
    df=pd.read_csv(i)
    t=df.groupby(by=['id_cohort', 'dosage'], as_index=False).sum()
    t['bar_ori_idx']=t['ori_trial']/t['bin_trial_n_sum']
    plot=sns.barplot(data=t, x='dosage', y='bar_ori_idx', order=["AMPH-vehicle", "AMPH-0.3mpk", "AMPH-0.6mpk","AMPH-1mpk"], 
                       palette=["black", "green", "royalblue", "red"],capsize=.01, errorbar=('ci', 68) )
    title="%s_amph_effect_on_total_orientation_index_in_LH"%(x)
    t.to_csv(save_path+"\\"+title+".csv")

    plt.savefig(save_path+"\%s.png"%(title), dpi=800, bbox_inches="tight")
    plt.close()
    